<a href="https://colab.research.google.com/github/niuhaoyu16/Pytorch/blob/master/Define_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F




In [3]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1,6,3)
    self.conv2 = nn.Conv2d(6,16,3)

    self.fc1 = nn.Linear(16*6*6,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    # max pooling
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]
    num_features = 1
    for s in size:
      num_features *=s
    return num_features

net = Net()
print(net)



Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [7]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0113,  0.0530,  0.1036, -0.0683,  0.0266,  0.0538, -0.0357, -0.0725,
          0.0378, -0.0046]], grad_fn=<AddmmBackward>)


In [0]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [10]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6742, grad_fn=<MseLossBackward>)


In [11]:
loss.grad_fn

In [12]:
loss.grad_fn.next_functions[0][0]

In [13]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]

In [0]:
net.zero_grad()


In [15]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [0]:
loss.backward()

In [17]:
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([ 0.0083, -0.0003, -0.0059,  0.0089, -0.0144,  0.0017])


In [0]:
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

In [0]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(),lr=0.01)

# in your training loop
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()